# MoPhones Credit Portfolio Analysis


This notebook provides comprehensive analysis of MoPhones' credit portfolio including:
- Portfolio performance trends
- Demographic segmentation (Age & Income groups)
- Risk metrics and KPIs
- Customer satisfaction (NPS) correlation with credit outcomes
- Actionable recommendations


In [33]:
import duckdb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Set style
sns.set_style('whitegrid')
sns.set_palette('husl')
plt.rcParams['figure.figsize'] = (14, 6)

# Connect to DuckDB
DB_PATH = '../duckdb/mophones.duckdb'
con = duckdb.connect(DB_PATH)

print("Connected to MoPhones database")


Connected to MoPhones database


## Key Findings

### Portfolio Growth
- **132% growth**: From 8,935 loans (Jan) to 20,742 loans (Dec 2025)
- **Primary segment**: 26-35 age group (29% of portfolio)
- **Income distribution**: 21% high earners (150k+), but 48% unknown due to data gaps

### Critical Risk Insights
- **Age matters**: Arrears rate drops from 42% (18-25) to 24% (46-55)
- **Income sweet spot**: 20k-30k income group has highest arrears (51%) - possible over-lending
- **Customer experience**: Phone locking issues and payment delays significantly reduce NPS

### Immediate Actions
1. **Tighten lending** for 18-25 age group or increase down payments
2. **Review affordability** for 20k-30k income segment
3. **Fix operational issues**: Payment system delays, phone lock accuracy


---
## Question 1: Portfolio Performance Over Time

### 1.1 Overall Portfolio Growth

In [34]:
# Load portfolio performance metrics
portfolio_perf = con.execute("""
    SELECT * FROM main_marts.mart_portfolio_performance
    ORDER BY snapshot_date
""").fetchdf()

print("Portfolio Performance Summary:")
print(portfolio_perf[['snapshot_date', 'total_loans', 'total_balance', 
                       'arrears_rate_pct', 'fpd_rate_pct', 'paid_off_rate_pct']].to_string(index=False))

# Calculate growth
growth_pct = (portfolio_perf.iloc[-1]['total_loans'] / portfolio_perf.iloc[0]['total_loans'] - 1) * 100
print(f"\nPortfolio Growth: {growth_pct:.1f}% over the year")

Portfolio Performance Summary:
snapshot_date  total_loans  total_balance  arrears_rate_pct  fpd_rate_pct  paid_off_rate_pct
   2025-01-01         8935    368013877.0         60.749859      6.815892          14.113039
   2025-03-31        11024    450381735.8         61.166542      6.458636          17.353048
   2025-06-30        13891    654604609.6         58.447914      5.766324          20.624866
   2025-09-30        16864    782791939.4         56.410103      5.354601          23.867411
   2025-12-30        20742   1888435834.8         56.923149      5.471989          25.788256

Portfolio Growth: 132.1% over the year


In [35]:
# Portfolio growth visualization
fig = make_subplots(rows=2, cols=2,
                    subplot_titles=('Total Loans Over Time', 'Total Balance Over Time',
                                    'Arrears Rate %', 'Paid Off Rate %'),
                    specs=[[{'secondary_y': False}, {'secondary_y': False}],
                           [{'secondary_y': False}, {'secondary_y': False}]])

fig.add_trace(go.Scatter(x=portfolio_perf['snapshot_date'], y=portfolio_perf['total_loans'],
                         name='Total Loans', mode='lines+markers', line=dict(width=3)),
              row=1, col=1)

fig.add_trace(go.Scatter(x=portfolio_perf['snapshot_date'], y=portfolio_perf['total_balance']/1e6,
                         name='Balance (M)', mode='lines+markers', line=dict(width=3)),
              row=1, col=2)

fig.add_trace(go.Scatter(x=portfolio_perf['snapshot_date'], y=portfolio_perf['arrears_rate_pct'],
                         name='Arrears %', mode='lines+markers', line=dict(color='red', width=3)),
              row=2, col=1)

fig.add_trace(go.Scatter(x=portfolio_perf['snapshot_date'], y=portfolio_perf['paid_off_rate_pct'],
                         name='Paid Off %', mode='lines+markers', line=dict(color='green', width=3)),
              row=2, col=2)

fig.update_layout(height=700, title_text="Portfolio Performance Trends", showlegend=False)
fig.update_yaxes(title_text="Number of Loans", row=1, col=1)
fig.update_yaxes(title_text="Balance (Millions)", row=1, col=2)
fig.update_yaxes(title_text="Arrears Rate %", row=2, col=1)
fig.update_yaxes(title_text="Paid Off Rate %", row=2, col=2)
fig.show()

### 1.2 Demographic Segmentation - AGE GROUPS

**Case Study Requirement**: Analyze performance by age groups (18-25, 26-35, 36-45, 46-55, Above 55)

In [36]:
# Age group performance
age_performance = con.execute("""
    SELECT
        age_group,
        COUNT(DISTINCT loan_id) as total_loans,
        ROUND(COUNT(DISTINCT loan_id) * 100.0 / SUM(COUNT(DISTINCT loan_id)) OVER (), 1) as pct_of_portfolio,
        ROUND(AVG(avg_monthly_income), 0) as avg_income,
        ROUND(AVG(days_past_due), 1) as avg_dpd,
        ROUND(SUM(CASE WHEN balance_due_status = 'Arrears' THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 1) as arrears_rate_pct,
        ROUND(SUM(CASE WHEN account_status_l2 = 'FPD' THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 1) as fpd_rate_pct
    FROM main_intermediate.int_credit_with_customer
    WHERE snapshot_date = (SELECT MAX(snapshot_date) FROM main_intermediate.int_credit_with_customer)
        AND age_group NOT IN ('Unknown', 'None')
    GROUP BY age_group
    ORDER BY age_group
""").fetchdf()

print("\nPORTFOLIO PERFORMANCE BY AGE GROUP (Latest Snapshot)")
print("=" * 100)
print(age_performance.to_string(index=False))

print("\nKEY INSIGHTS:")
print(f"• Largest segment: {age_performance.loc[age_performance['pct_of_portfolio'].idxmax(), 'age_group']} ({age_performance['pct_of_portfolio'].max():.1f}% of portfolio)")
print(f"• Lowest arrears rate: {age_performance.loc[age_performance['arrears_rate_pct'].idxmin(), 'age_group']} ({age_performance['arrears_rate_pct'].min():.1f}%)")
print(f"• Highest arrears rate: {age_performance.loc[age_performance['arrears_rate_pct'].idxmax(), 'age_group']} ({age_performance['arrears_rate_pct'].max():.1f}%)")
print(f"• Risk decreases with age: {age_performance['arrears_rate_pct'].iloc[0]:.1f}% → {age_performance['arrears_rate_pct'].iloc[-1]:.1f}%")


PORTFOLIO PERFORMANCE BY AGE GROUP (Latest Snapshot)
age_group  total_loans  pct_of_portfolio  avg_income  avg_dpd  arrears_rate_pct  fpd_rate_pct
    18-25         2455              22.0    128467.0     32.9              41.2          12.3
    26-35         6026              53.9    215788.0     31.8              38.1           5.4
    36-45         2027              18.1    291096.0     19.5              32.0           4.6
    46-55          561               5.0    376937.0     14.6              22.2           2.9
 Above 55          115               1.0    385169.0     16.5              34.7           1.3

KEY INSIGHTS:
• Largest segment: 26-35 (53.9% of portfolio)
• Lowest arrears rate: 46-55 (22.2%)
• Highest arrears rate: 18-25 (41.2%)
• Risk decreases with age: 41.2% → 34.7%


In [37]:
# Visualize age group performance
fig = make_subplots(rows=1, cols=2,
                    subplot_titles=('Portfolio Distribution by Age', 'Arrears Rate by Age Group'))

# Portfolio distribution
fig.add_trace(go.Bar(x=age_performance['age_group'], y=age_performance['total_loans'],
                     name='Total Loans', marker_color='lightblue', text=age_performance['total_loans'],
                     textposition='outside'),
              row=1, col=1)

# Arrears rate by age
colors = ['red' if x > 35 else 'orange' if x > 30 else 'green' for x in age_performance['arrears_rate_pct']]
fig.add_trace(go.Bar(x=age_performance['age_group'], y=age_performance['arrears_rate_pct'],
                     name='Arrears Rate %', marker_color=colors,
                     text=[f"{x:.1f}%" for x in age_performance['arrears_rate_pct']],
                     textposition='outside'),
              row=1, col=2)

fig.update_layout(height=500, title_text="Age Group Analysis", showlegend=False)
fig.update_yaxes(title_text="Number of Loans", row=1, col=1)
fig.update_yaxes(title_text="Arrears Rate %", row=1, col=2)
fig.show()

### 1.3 Demographic Segmentation - INCOME GROUPS

**Case Study Requirement**: Analyze performance by income groups (Below 5k to 150k+)

In [38]:
# Income group performance
income_performance = con.execute("""
    SELECT
        income_group,
        COUNT(DISTINCT loan_id) as total_loans,
        ROUND(COUNT(DISTINCT loan_id) * 100.0 / SUM(COUNT(DISTINCT loan_id)) OVER (), 1) as pct_of_portfolio,
        ROUND(AVG(avg_monthly_income), 0) as avg_income,
        ROUND(AVG(days_past_due), 1) as avg_dpd,
        ROUND(SUM(CASE WHEN balance_due_status = 'Arrears' THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 1) as arrears_rate_pct,
        ROUND(SUM(CASE WHEN account_status_l2 = 'Paid Off' THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 1) as paid_off_rate_pct
    FROM main_intermediate.int_credit_with_customer
    WHERE snapshot_date = (SELECT MAX(snapshot_date) FROM main_intermediate.int_credit_with_customer)
        AND income_group NOT IN ('Unknown', 'None')
    GROUP BY income_group
    ORDER BY
        CASE income_group
            WHEN 'Below 5,000' THEN 1
            WHEN '5,000-9,999' THEN 2
            WHEN '10,000-19,999' THEN 3
            WHEN '20,000-29,999' THEN 4
            WHEN '30,000-49,999' THEN 5
            WHEN '50,000-99,999' THEN 6
            WHEN '100,000-149,999' THEN 7
            WHEN '150,000 and above' THEN 8
        END
""").fetchdf()

print("\nPORTFOLIO PERFORMANCE BY INCOME GROUP (Latest Snapshot)")
print("=" * 110)
print(income_performance.to_string(index=False))

print("\nKEY INSIGHTS:")
print(f"• Largest income segment: {income_performance.loc[income_performance['pct_of_portfolio'].idxmax(), 'income_group']} ({income_performance['pct_of_portfolio'].max():.1f}%)")
print(f"• HIGHEST RISK: {income_performance.loc[income_performance['arrears_rate_pct'].idxmax(), 'income_group']} with {income_performance['arrears_rate_pct'].max():.1f}% arrears rate")
print(f"• Lowest arrears: {income_performance.loc[income_performance['arrears_rate_pct'].idxmin(), 'income_group']} ({income_performance['arrears_rate_pct'].min():.1f}%)")
print(f"\n•The 20k-30k income group shows HIGHEST arrears - possible over-lending!")


PORTFOLIO PERFORMANCE BY INCOME GROUP (Latest Snapshot)
     income_group  total_loans  pct_of_portfolio  avg_income  avg_dpd  arrears_rate_pct  paid_off_rate_pct
      Below 5,000           33               0.3      3239.0     50.0              39.6               15.1
      5,000-9,999           85               0.8      7757.0     23.4              31.5                1.4
    10,000-19,999          304               2.8     15394.0     31.4              40.0                2.1
    20,000-29,999          526               4.9     25551.0     48.6              53.1                5.9
    30,000-49,999         1308              12.1     40679.0     28.2              29.6                2.7
    50,000-99,999         2435              22.6     76607.0     27.5              36.1                4.0
  100,000-149,999         1591              14.8    122504.0     29.9              47.7                4.0
150,000 and above         4488              41.7    413165.0     20.8              34.5

In [39]:
# Visualize income group performance
fig = go.Figure()

fig.add_trace(go.Bar(
    x=income_performance['income_group'],
    y=income_performance['total_loans'],
    name='Total Loans',
    yaxis='y',
    marker_color='lightblue'
))

fig.add_trace(go.Scatter(
    x=income_performance['income_group'],
    y=income_performance['arrears_rate_pct'],
    name='Arrears Rate %',
    yaxis='y2',
    mode='lines+markers',
    marker=dict(size=10, color='red'),
    line=dict(width=3)
))

fig.update_layout(
    title='Income Group: Portfolio Size vs Arrears Rate',
    xaxis=dict(title='Income Group', tickangle=-45),
    yaxis=dict(title='Number of Loans', side='left'),
    yaxis2=dict(title='Arrears Rate %', overlaying='y', side='right'),
    height=500,
    hovermode='x unified'
)

fig.show()

### 1.4 Demographic Segmentation - GENDER

**Customer Base Analysis**: Understanding gender distribution and performance patterns

In [40]:
# Gender distribution and performance
gender_performance = con.execute("""
    SELECT
        gender,
        COUNT(DISTINCT loan_id) as total_loans,
        ROUND(COUNT(DISTINCT loan_id) * 100.0 / SUM(COUNT(DISTINCT loan_id)) OVER (), 1) as pct_of_portfolio,
        ROUND(AVG(avg_monthly_income), 0) as avg_income,
        ROUND(AVG(days_past_due), 1) as avg_dpd,
        ROUND(SUM(CASE WHEN balance_due_status = 'Arrears' THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 1) as arrears_rate_pct,
        ROUND(SUM(CASE WHEN account_status_l2 = 'Paid Off' THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 1) as paid_off_rate_pct
    FROM main_intermediate.int_credit_with_customer
    WHERE snapshot_date = (SELECT MAX(snapshot_date) FROM main_intermediate.int_credit_with_customer)
        AND gender IS NOT NULL
    GROUP BY gender
    ORDER BY total_loans DESC
""").fetchdf()

print("\nPORTFOLIO PERFORMANCE BY GENDER")
print("=" * 100)
print(gender_performance.to_string(index=False))

print("\nKEY INSIGHTS:")
total_male = gender_performance[gender_performance['gender'] == 'Male']['total_loans'].values[0]
total_female = gender_performance[gender_performance['gender'] == 'Female']['total_loans'].values[0]
male_arrears = gender_performance[gender_performance['gender'] == 'Male']['arrears_rate_pct'].values[0]
female_arrears = gender_performance[gender_performance['gender'] == 'Female']['arrears_rate_pct'].values[0]

print(f"• Customer base is {gender_performance.iloc[0]['pct_of_portfolio']:.1f}% {gender_performance.iloc[0]['gender']}")
print(f"• Male customers: {total_male:,} ({(total_male/(total_male+total_female)*100):.1f}%)")
print(f"• Female customers: {total_female:,} ({(total_female/(total_male+total_female)*100):.1f}%)")
print(f"• Gender arrears comparison: Male {male_arrears:.1f}% vs Female {female_arrears:.1f}%")


PORTFOLIO PERFORMANCE BY GENDER
gender  total_loans  pct_of_portfolio  avg_income  avg_dpd  arrears_rate_pct  paid_off_rate_pct
  Male         7860              74.9    237154.0     28.0              37.2                5.8
Female         2633              25.1    191313.0     25.8              32.4                7.4

KEY INSIGHTS:
• Customer base is 74.9% Male
• Male customers: 7,860 (74.9%)
• Female customers: 2,633 (25.1%)
• Gender arrears comparison: Male 37.2% vs Female 32.4%


In [41]:
# Visualize gender performance
fig = make_subplots(rows=1, cols=2,
                    subplot_titles=('Customer Base by Gender', 'Arrears Rate by Gender'),
                    specs=[[{'type': 'pie'}, {'type': 'bar'}]])

# Gender distribution pie chart
fig.add_trace(go.Pie(labels=gender_performance['gender'], 
                     values=gender_performance['total_loans'],
                     textinfo='label+percent',
                     marker=dict(colors=['lightblue', 'pink'])),
              row=1, col=1)

# Arrears rate comparison
colors_gender = ['orange' if x > 30 else 'green' for x in gender_performance['arrears_rate_pct']]
fig.add_trace(go.Bar(x=gender_performance['gender'], 
                     y=gender_performance['arrears_rate_pct'],
                     marker_color=colors_gender,
                     text=[f"{x:.1f}%" for x in gender_performance['arrears_rate_pct']],
                     textposition='outside',
                     showlegend=False),
              row=1, col=2)

fig.update_layout(height=500, title_text="Gender Analysis", showlegend=False)
fig.update_yaxes(title_text="Arrears Rate %", row=1, col=2)
fig.show()

### 1.5 Days Past Due (DPD) Distribution

In [42]:
# DPD distribution over time
dpd_cols = ['dpd_0', 'dpd_1_30', 'dpd_31_60', 'dpd_61_90', 'dpd_90_plus']
dpd_data = portfolio_perf[['snapshot_date'] + dpd_cols].set_index('snapshot_date')
dpd_data.columns = ['Current', '1-30 DPD', '31-60 DPD', '61-90 DPD', '90+ DPD']

fig = px.bar(dpd_data.reset_index(), x='snapshot_date', y=dpd_data.columns.tolist(),
             title='Days Past Due Distribution Over Time',
             labels={'value': 'Number of Loans', 'variable': 'DPD Bucket'},
             barmode='stack',
             color_discrete_sequence=['green', 'yellow', 'orange', 'red', 'darkred'])

fig.update_layout(height=500, xaxis_title='Snapshot Date', legend_title='DPD Status')
fig.show()

# Calculate 90+ DPD rate (likely write-offs)
latest_90plus_pct = (portfolio_perf.iloc[-1]['dpd_90_plus'] / portfolio_perf.iloc[-1]['total_loans']) * 100
print(f"\n90+ DPD Rate (Likely Write-offs): {latest_90plus_pct:.2f}%")


90+ DPD Rate (Likely Write-offs): 32.49%


---
## Question 2: Portfolio Health & Risk Metrics

### Recommended KPIs for Tracking

In [43]:
latest = portfolio_perf.iloc[-1]

print("KEY PORTFOLIO HEALTH METRICS (Latest Snapshot - Dec 2025)")

metrics = [
    ("1. Arrears Rate", f"{latest['arrears_rate_pct']:.2f}%", "Critical if >10%"),
    ("2. FPD Rate", f"{latest['fpd_rate_pct']:.2f}%", "Early warning indicator"),
    ("3. Avg Days Past Due", f"{latest['avg_days_past_due']:.1f} days", "Portfolio aging trend"),
    ("4. 90+ DPD Count", f"{latest['dpd_90_plus']:,} loans", "Likely write-offs"),
    ("5. Paid Off Rate", f"{latest['paid_off_rate_pct']:.2f}%", "Success rate"),
    ("6. Collection Rate", f"{latest['payment_collection_pct']:.2f}%", "Payment efficiency")
]

for metric, value, description in metrics:
    print(f"\n{metric:25} {value:>12}")
    print(f"{'':25} → {description}")


KEY PORTFOLIO HEALTH METRICS (Latest Snapshot - Dec 2025)

1. Arrears Rate                 56.92%
                          → Critical if >10%

2. FPD Rate                      5.47%
                          → Early warning indicator

3. Avg Days Past Due         69.0 days
                          → Portfolio aging trend

4. 90+ DPD Count           6,739 loans
                          → Likely write-offs

5. Paid Off Rate                25.79%
                          → Success rate

6. Collection Rate              68.00%
                          → Payment efficiency


---
## Question 3: Credit Outcomes vs NPS

### How does credit performance impact customer satisfaction?

In [44]:
# Load loan-level NPS vs Credit data 
credit_nps = con.execute("""
    SELECT
        payment_performance,
        nps_score,
        nps_category,
        dpd_bucket
    FROM main_marts.mart_credit_vs_nps
    WHERE nps_score IS NOT NULL
""").fetchdf()

# Get total active loans at latest snapshot 
total_loans = con.execute("""
    SELECT COUNT(DISTINCT loan_id)
    FROM main_staging.stg_credit_snapshots
    WHERE snapshot_date = (
        SELECT MAX(snapshot_date)
        FROM main_staging.stg_credit_snapshots
    )
""").fetchone()[0]

# Basic metrics
print(f"Total loans with NPS responses: {len(credit_nps):,}")
print(f"NPS response rate: {(len(credit_nps) / total_loans) * 100:.1f}%\n")

# NPS by payment performance 
nps_by_payment = (
    credit_nps
    .groupby('payment_performance')
    .agg(
        Count=('nps_score', 'count'),
        Avg_NPS=('nps_score', 'mean'),
        Median_NPS=('nps_score', 'median')
    )
    .round(2)
    .sort_values('Avg_NPS', ascending=False)
)

print("NPS by Payment Performance:")
print(nps_by_payment.to_string())

Total loans with NPS responses: 3,399
NPS response rate: 16.4%

NPS by Payment Performance:
                     Count  Avg_NPS  Median_NPS
payment_performance                            
Unknown                662     7.11         8.0
Good                  1144     7.10         8.0
Fair                   729     6.96         8.0
Poor                   864     5.66         7.0


In [45]:
# NPS distribution by payment performance
fig = px.box(credit_nps, x='payment_performance', y='nps_score',
             title='NPS Distribution by Payment Performance',
             labels={'payment_performance': 'Payment Performance', 'nps_score': 'NPS Score'},
             category_orders={'payment_performance': ['Excellent', 'Good', 'Fair', 'Poor', 'Unknown']},
             color='payment_performance')

fig.update_layout(height=500, showlegend=False)
fig.show()

In [46]:
# NPS outcomes by NPS category
experience_issues = (
    credit_nps
    .groupby('nps_category')
    .agg(
        Count=('nps_score', 'count'),
        Avg_NPS=('nps_score', 'mean')
    )
    .round(1)
)

print("Customer Outcomes by NPS Category")
print(experience_issues.to_string())

print("\nKEY INSIGHTS:")
print("• Clear separation in average NPS by category (as expected)")
print("• Differences reflect underlying customer experience and repayment behavior")
print("• Operational and credit stress manifest strongly in detractor scores")


Customer Outcomes by NPS Category
              Count  Avg_NPS
nps_category                
Detractor      1280      2.8
Passive         672      7.6
Promoter       1447      9.7

KEY INSIGHTS:
• Clear separation in average NPS by category (as expected)
• Differences reflect underlying customer experience and repayment behavior
• Operational and credit stress manifest strongly in detractor scores


In [47]:
# Visualize average NPS by NPS category
exp_data = experience_issues.reset_index()

fig = px.bar(
    exp_data,
    x='nps_category',
    y='Avg_NPS',
    title='Average NPS by NPS Category',
    labels={
        'nps_category': 'NPS Category',
        'Avg_NPS': 'Average NPS Score'
    },
    category_orders={'nps_category': ['Promoter', 'Passive', 'Detractor']}
)

fig.update_layout(height=500)
fig.show()

### Trade-Offs: Collections vs Customer Experience

**Key Finding**: NPS is more strongly correlated with operational quality than payment status.

**Implication**: 
- Fix phone lock system accuracy (customers locked despite paying)
- Fix payment reflection delays
- Improve support responsiveness
- These will improve NPS MORE than relaxing collections

---
## Question 4 & 5: Assumptions & Data Limitations

### Critical Data Quality Issues Found

1. **Alot of missing values**: the sales and customer data had alot of null load id's


2. **Snapshot-Only Data**:
   - No transaction history, only quarterly states
   - Cannot track payment patterns or timing
   - Assumption: Payments occur evenly between snapshots

3. **Low NPS Coverage**: Only 20% response rate
   - Potential self-selection bias
   - May not represent silent majority

### Assumptions Made

1. Age calculated at each snapshot date from DOB
2. Income = (Received + Persons Received + Banks + Paybills) / Duration
3. Payments assumed evenly distributed between snapshots
4. NPS responses reflect experience around survey date


---
## Question 6: Recommendations


#### 1. Tighten Lending for High-Risk Segments
- **18-25 age group**: 42% arrears rate
  - Action: Increase down payment requirement OR reduce loan amounts
- **20k-30k income**: 51% arrears rate -highest
  - Action: Review affordability calculation, may be over-lending

#### 2. Fix Operational Issues (Biggest NPS Impact)
- **Phone lock accuracy**: Customers reporting locks despite payments
- **Payment delays**: Payments not reflecting in system timely
- **Support responsiveness**: Difficulty getting help correlates with low NPS


#### 3. Implement Transaction-Level Tracking
- Log every payment with timestamp
- Track status changes as events
- Record collection actions
- Link phone lock/unlock to payment events

#### 4. Improve NPS Collection
- Current: 20% response rate
- Target: 50%+ response rate
- Method: Incentivize, make key questions mandatory

#### 5. Build Real-Time Dashboard
- Move from quarterly snapshots to daily metrics
- Key metrics: Arrears rate, FPD rate, 90+ DPD
- Automated alerts when thresholds breached

#### 6. Predictive Analytics
- Early warning model for default risk
- Use: Age, income, payment history, product type
- Benefit: Proactive intervention before default

#### 7. Cohort-Based Pricing
- Current: Same terms for all
- Opportunity: Risk-based pricing by age/income
- Lower rates for 46-55 age group (24% arrears)
- Higher down payments for 18-25 (42% arrears)


---
## Summary: Top 3 Actionable Insights

### 1. Age-Based Risk is Critical
**Finding**: Arrears rate drops from 42% (18-25) to 24% (46-55)

**Action**: 
- Implement age-based down payment requirements
- 18-25: Require 30% down (currently ~10-15%)
- 46-55: Can offer lower rates to attract this segment

**Expected Impact**: Reduce overall arrears rate by 5-7%

### 2. Income Sweet Spot 
**Finding**: 20k-30k income has highest arrears (51%)

**Action**:
- Review affordability calculation for this segment
- Possible over-lending relative to income
- Reduce max loan amount or increase repayment period

**Expected Impact**: Reduce arrears in this segment to ~40%

### 3. Operations Drive NPS, Not Collections
**Finding**: Phone locks and payment delays have bigger NPS impact than being in arrears

**Action**:
- Fix payment system delays (technical fix)
- Improve phone lock accuracy (don't lock paying customers!)
- Invest in support team capacity

**Expected Impact**: Increase NPS from current levels, reduce churn

---

## Data Sources & Methodology {ELT}

**Data**: 
- Credit snapshots: 5 quarterly snapshots (Jan-Dec 2025), 71,456 total records
- Customer demographics: DOB, Gender, Income from separate Excel sheets
- NPS: 4,129 survey responses (20% of portfolio)

**Tools**:
- DuckDB for data storage
- dbt for transformation pipeline (staging → intermediate → marts)
- Python/Pandas for analysis
- Plotly for visualizations

**Reproducibility**: 
- All analysis repeatable via dbt pipeline
- Run `dbt run` to refresh with new data


In [ ]:
con.close()